In [ ]:
!pip install tensorboardX



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 13.7 MB/s 


In [ ]:
!pip install gym
!pip install Box2D
!pip install pygame

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 13.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 21.8 MB 1.3 MB/s 


https://keras.io/ko/initializers/ (참조링크) 

1. Critic Model의 Initializer를 glorot_uniform, lecun_uniform로 바꾸어 테스트 진행
2. Actor Model의 Initializer를 RandomNormal에서, stddev의 값을 10의 거듭 제곱 단위로 바꿔 3개 set 학습 진행 

Actormodel 학습 진행하고, Critic model 학습 진행하고, 두 개 최적값을 합해서 학습 진행 

※ 이 때, 아마 log 데이터가 2개 폴터 쌓일텐데, 반드시 설정한 변수값 

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import random
import gym
import pylab
import numpy as np
import copy
from gym.wrappers import GrayScaleObservation
from gym.spaces import Box
from gym import Wrapper
import time
# Deep neural network 학습을 위한 tensorflow 
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import backend as K
import tensorflow as tf
from tensorboardX import SummaryWriter
tf.compat.v1.disable_eager_execution() 


# GPU를 사용 가능하도록 설정해줌. (있다면)
gpus = tf.config.experimental.list_physical_devices('GPU')
if len(gpus) > 0:
    print(f'GPUs {gpus}')
    try: tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError: pass



class ActorModel:
    def __init__(self, lr, optimizer, inputShape, actionSpace):
        input = Input(inputShape)
        self.actionSpace = actionSpace
        layer = Dense(128, activation="relu", kernel_initializer=tf.random_normal_initializer(stddev=0.1))(input)
        layer = Dense(64, activation="relu", kernel_initializer=tf.random_normal_initializer(stddev=0.1))(layer)
        output = Dense(actionSpace, activation="softmax")(layer) # flatten 후, 최종적인 output을 action space의 shape에 맞게 출력하도록함.

        self.Actor = Model(inputs = input, outputs = output) #OUTPUT 값으로써 각 행동들을 취할 확률을 리턴한다. 
        self.Actor.compile(loss=self.loss, optimizer=optimizer(lr=lr))

    # Actor model의 ppo loss를 tensorflow 모델로 넘겨주는 함수. 
    def loss(self, y_true, y_pred):
        advantages, prediction_picks, actions = y_true[:, :1], y_true[:, 1:1+self.actionSpace], y_true[:, 1+self.actionSpace:]
        LOSS_CLIPPING = 0.05
        ENTROPY_LOSS = 0.001

        oldProb = actions * prediction_picks
        newProb = actions * y_pred

        oldProb = K.clip(oldProb, 1e-10, 1.0)
        newProb = K.clip(newProb, 1e-10, 1.0)
        ratio = K.exp(K.log(newProb) - K.log(oldProb))
        p1 = ratio * advantages
        p2 = K.clip(ratio, min_value=1 - LOSS_CLIPPING, max_value=1 + LOSS_CLIPPING) * advantages

        actor_loss = -K.mean(K.minimum(p1, p2))
        entropy = -(y_pred * K.log(y_pred + 1e-10))
        entropy = ENTROPY_LOSS * K.mean(entropy)
        
        total = actor_loss - entropy
        return total

    def predict(self, state):
        return self.Actor.predict(state)

class CriticModel:
    def __init__(self,  lr, optimizer, inputShape, actionSpace):
        input = Input(inputShape) # state,reward 값이 들어옴
        oldValue = Input(shape=(1,)) # 과거 실행한 state 값을 의미

        layer = Dense(128, activation="relu", kernel_initializer='he_uniform')(input)
        layer = Dense(64, activation="relu", kernel_initializer='he_uniform')(layer)
        value = Dense(1, activation=None)(layer)

        self.Critic = Model(inputs=[input, oldValue], outputs = value) # output 값으로써 Q_VALUE가 리턴된다. 
        self.Critic.compile(loss=[self.critic_PPO2_loss(oldValue)], optimizer=optimizer(lr=lr))

    def critic_PPO2_loss(self, values):
        def loss(y_true, y_pred):
            LOSS_CLIPPING = 0.05
            clipped_v_loss = values + K.clip(y_pred - values, -LOSS_CLIPPING, LOSS_CLIPPING)
            valueLoss = K.mean((y_true - y_pred) ** 2) 
            return valueLoss
        return loss

    def predict(self, state):
        return self.Critic.predict([state, np.zeros((state.shape[0], 1))])

class PPOAgent:
    def __init__(self):
        self.env_name = 'LunarLander-v2'
        self.env = gym.make(self.env_name)
        self.state_size = self.env.observation_space.shape
        self.action_size = self.env.action_space.n
        self.lr = 0.001 # 설정한 learning rate 
        self.epochs = 10 # training epochs
        self.shuffle=False
        self.train_batch = 1000
        self.EPISODES = 10000 # 총 학습시킬 episode 타깃 수 
        self.current_epi = 0 # 현재 몇 개의 episode를 학습시켰는지를 저장해주는 변수 
        self.max_avg = 0 # 현재 최고 몇 점이 났는지를 저장해두는 변수 
        self.optimizer = Adam
        self.count_replay = 0
        self.writer = SummaryWriter(comment="_"+self.env_name+"_"+self.optimizer.__name__+"_"+str(self.lr))
        
        # matplot을 쓰기 위한 메모리 초기화 
        self.scores_, self.episodes_, self.average_ = [], [], [] 

        # Actor-Critic 모델을 생성함 
        self.Actor = ActorModel(lr=self.lr, optimizer = self.optimizer, inputShape=self.state_size, actionSpace = self.action_size)
        self.Critic = CriticModel( lr=self.lr, optimizer = self.optimizer, inputShape=self.state_size, actionSpace = self.action_size)
        
        self.ActorName = f"{self.env_name}_PPO_Actor.h5" # 모델 파일을 저장하기 위한 변수. 
        self.CriticName = f"{self.env_name}_PPO_Critic.h5"

    # Actor모델을 통해 예측한 값을 리턴해주는 함수입니다. 
    def act(self, state):
        prediction = self.Actor.predict(state)[0] # 예측값 배열을 가져옴 
        action = np.random.choice(self.action_size, p=prediction) # action들이 뽑힐 확률을 지정해준후, 뽑기. 
        action_one_hot = np.zeros([self.action_size])
        action_one_hot[action] = 1 # 뽑힌 액션에 대해 1표시 by one hot encoding
        return action, action_one_hot, prediction

    def get_gaes(self, rewards, dones, values, next_values, gamma = 0.99, lamda = 0.9, normalize=True):
      # 에피소드가 지날수록 gamma 값이 곱해진 reward가 리턴됨. 
        deltas = [r + gamma * (1 - d) * nv - v for r, d, nv, v in zip(rewards, dones, next_values, values)]
        deltas = np.stack(deltas)
        gaes = copy.deepcopy(deltas)
        for t in reversed(range(len(deltas) - 1)):
            gaes[t] = gaes[t] + (1 - dones[t]) * gamma * lamda * gaes[t + 1]

        target = gaes + values
        if normalize:
            gaes = (gaes - gaes.mean()) / (gaes.std() + 1e-8)
        return np.vstack(gaes), np.vstack(target)

    # 지난 Action을 Critic model이 평가 후, 이를 보정해주는 알고리즘이 담긴 함수.
    # 적절한 학습을 위해 np.vstack을 통해 input shape에 맞는 형태로 바꾸어  input으로 전달.  
    def replay(self, states, actions, rewards, predictions, dones, next_states):
        states = np.vstack(states)
        next_states = np.vstack(next_states)
        actions = np.vstack(actions)
        predictions = np.vstack(predictions)

        # Critic method를 통해 reward 값을 얻어옴 
        values = self.Critic.predict(states)
        next_values = self.Critic.predict(next_states)

        #GAE 보정을 통한 Reward값을 저장. 
        advantages, target = self.get_gaes(rewards, dones, np.squeeze(values), np.squeeze(next_values)) 

        # loss 함수에 쓰이기 적합하도록 배열을 가로로 결합하여줌. 
        y_true = np.hstack([advantages, predictions, actions])
        
        # Actor와 Critic 모델을 학습시켜줌. 
        actor_loss = self.Actor.Actor.fit(states, y_true, epochs=self.epochs, verbose=0, shuffle=self.shuffle)
        critic_loss = self.Critic.Critic.fit([states, values], target, epochs=self.epochs, verbose=0, shuffle=self.shuffle)

        self.writer.add_scalar('Data/actor_loss_per_replay', np.sum(actor_loss.history['loss']), self.count_replay)
        self.writer.add_scalar('Data/critic_loss_per_replay', np.sum(critic_loss.history['loss']), self.count_replay)
        self.count_replay += 1
        
    def load(self):
        self.Actor.Actor.load_weights(self.ActorName)
        self.Critic.Critic.load_weights(self.CriticName)

    def save(self):
        self.Actor.Actor.save_weights(self.ActorName)
        self.Critic.Critic.save_weights(self.CriticName)
        
    pylab.figure(figsize=(18, 9))
    pylab.subplots_adjust(left=0.05, right=0.98, top=0.96, bottom=0.06)
    def PlotModel(self, score, current_epi): 
        self.scores_.append(score)
        self.episodes_.append(current_epi)
        self.average_.append(sum(self.scores_[-50:]) / len(self.scores_[-50:])) 
        if str(current_epi)[-2:] == "00":# much faster than episode % 100
            pylab.plot(self.episodes_, self.scores_, 'b')
            pylab.plot(self.episodes_, self.average_, 'r')
            pylab.title(self.env_name+" PPO training cycle", fontsize=18)
            pylab.ylabel('Score', fontsize=18)
            pylab.xlabel('Steps', fontsize=18)
            try:
                pylab.grid(True)
                pylab.savefig(self.env_name+".png")
            except OSError:
                pass
        # saving best models
        if self.average_[-1] >= self.max_avg:
            self.max_avg = self.average_[-1]
            self.save()
            SAVING = "SAVING"
            # decreaate learning rate every saved model
            self.lr *= 0.95
            K.set_value(self.Actor.Actor.optimizer.learning_rate, self.lr)
            K.set_value(self.Critic.Critic.optimizer.learning_rate, self.lr)
        else:
            SAVING = ""

        return self.average_[-1], SAVING
    

    def run(self): # 매 batch마다 학습을 시행해주는 함수. 
        state = self.env.reset()
        state = np.reshape(state, [1, self.state_size[0]])
        done, score, SAVING = False, 0, ''
        while True:
            # ppo는 과거 값들도 참고하여 학습을 실시하므로, 이를 저장할 배열 선언.
            states, next_states, actions, rewards, predictions, dones = [], [], [], [], [], []
            for t in range(self.train_batch): # 설정해둔 MiniBatch 사이즈 만큼의 범위 내에서 학습을 진행함. 
                action, action_one_hot, prediction = self.act(state)
                
                next_state, reward, done, _ = self.env.step(action) # Actor가 예측한 action을 취해주고, 이를 env의 agent에게 전달. 
                # Critic모델에서 평가를 위해 next_state, reward 값은 저장해둠. 
                states.append(state) 
                next_states.append(np.reshape(next_state, [1, self.state_size[0]]))
                actions.append(action_one_hot) # ont hot encoding된 action값을 저장 
                rewards.append(reward)
                dones.append(done)
                predictions.append(prediction) 
                # 시행이 되었으므로, 현재 상태를 저장함. 이후 Critic 모델에 param으로 쓰임. 
                state = np.reshape(next_state, [1, self.state_size[0]])
                score += reward
                if done:
                    self.current_epi += 1
                    average, SAVING = self.PlotModel(score, self.current_epi) # 
                    print("episode: {}/{}, score: {}, average: {:.2f} {}".format(self.current_epi, self.EPISODES, score, average, SAVING))
                    self.writer.add_scalar(f'Workers:{1}/score_per_episode', score, self.current_epi)
                    self.writer.add_scalar(f'Workers:{1}/learning_rate', self.lr, self.current_epi)
                    

                    state, done, score, SAVING = self.env.reset(), False, 0, ''
                    state = np.reshape(state, [1, self.state_size[0]])
                    
            self.replay(states, actions, rewards, predictions, dones, next_states)
            if self.current_epi >= self.EPISODES:
                break
        self.env.close()  

    def test(self, test_episodes = 100):
        self.load()
        for e in range(100):
            state = self.env.reset()
            state = np.reshape(state, [1, self.state_size[0]])
            done = False
            score = 0
            while not done:
                self.env.render()
                action = np.argmax(self.Actor.predict(state)[0])
                state, reward, done, _ = self.env.step(action)
                state = np.reshape(state, [1, self.state_size[0]])
                score += reward
                if done:
                    print("episode: {}/{}, score: {}".format(e, test_episodes, score))
                    break
        self.env.close()


if __name__ == "__main__":
    model_train = PPOAgent()
    model_train.run() # train as PPO, train every epesode
   

/usr/local/lib/python3.7/dist-packages/gym/core.py:318: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
/usr/local/lib/python3.7/dist-packages/gym/wrappers/step_api_compatibility.py:40: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
/usr/local/lib/python3.7/dist-packages/ker

episode: 1/10000, score: -237.78052755013007, average: -237.78 
episode: 2/10000, score: -79.51147518057948, average: -158.65 
episode: 3/10000, score: -432.32362896118093, average: -249.87 
episode: 4/10000, score: -229.65117199554118, average: -244.82 
episode: 5/10000, score: -199.70669139891572, average: -235.79 
episode: 6/10000, score: -52.08370831546314, average: -205.18 
episode: 7/10000, score: -235.8897183526538, average: -209.56 
episode: 8/10000, score: -89.7164536009717, average: -194.58 
episode: 9/10000, score: -102.45434862507443, average: -184.35 
episode: 10/10000, score: -137.97074130111235, average: -179.71 
episode: 11/10000, score: -488.19852974130464, average: -207.75 
episode: 12/10000, score: -476.4481134548683, average: -230.14 
episode: 13/10000, score: -128.34994784615157, average: -222.31 
episode: 14/10000, score: -114.91977554378462, average: -214.64 
episode: 15/10000, score: -166.226064001679, average: -211.42 
episode: 16/10000, score: -432.49596292284

KeyboardInterrupt: ignored

<Figure size 1296x648 with 0 Axes>